In [1]:
import glob

In [2]:
dat = pd.read_pickle('./data/experiment_forcing_params_dynamic.pcl')
dat['idx'] = np.arange(1,len(dat)+1)

In [3]:
dat.tail(12)

,P,T,decade,krumlai,krumweight,name,scenarios,track,year,idx
0,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,1,PGW_MP,2000,115
1,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,2,PGW_MP,2000,116
2,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,3,PGW_MP,2000,117
3,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,1,PGW_MP,2000,118
4,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,2,PGW_MP,2000,119
5,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,3,PGW_MP,2000,120
0,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,1,PGW_LP,2000,121
1,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,2,PGW_LP,2000,122
2,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,5,PGW_LP,2000,123
3,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,1,PGW_LP,2000,124


In [4]:
dat.shape

(126, 10)

In [5]:
strt = '1993-10-1'
nd = '2013-09-30'

In [6]:
def parse_date(df):
    return '%i-%i-%i'%(df.year,df.month,df.day)

def readfile(fl):
    dat = pd.read_table(fl,sep=' ')
    dat['ET'] = dat.trans+dat.evap
    dat['datetime'] = dat.apply(parse_date,axis=1)
    dat.index = pd.DatetimeIndex(dat.datetime)
    dat['wateryear'] = dat.index.map(rut.wateryear)
    
    return dat[strt:nd]

In [49]:
def process(df):
    fl = df['file']
    track = '%s_%s'%(fl.split('_')[0].split('/')[-1],fl.split('_')[1])
    scenario = int(fl.split('_')[2]) # pull scenario
    year = int(fl.split('_')[-7]) # pull year
    decade = int(fl.split('_')[4]) # pull decade
    
    # pull the name of the simulation
    name1 = fl.split('_')[-5]
    name2 = fl.split('_')[-4]
    name3 = fl.split('_')[-3]
    name = '%s_%s_%s'%(name1,name2,name3)
    
    idx = int(fl.split('_')[-2]) # pull idx
    
    dat = readfile(fl)
    sfvals = dat.adj_streamflow.as_matrix()
    etvals = dat.ET.as_matrix()
    transvals = dat.trans.as_matrix()
    subvals = dat.snow_subl.as_matrix()
    snowfallvals = dat.snowfall.as_matrix()
    precipvals = dat.precip.as_matrix()
    evapvals = dat.evap.as_matrix()
    leakagevals = dat.leakage.as_matrix()
    rzs = dat.rz_storage.as_matrix()
    petvals = dat.pet.as_matrix()
    
    # compute some dataset whole values
    R_P = dat.adj_streamflow.sum()/dat.precip.sum()
    LAI = dat.lai.mean()
    ET = dat.ET.mean()
    leakage = dat.leakage.mean()
    
    
    # summarize by water year
    dat2 = dat.groupby(by='wateryear').mean()
    dat2['R_P'] = dat2.adj_streamflow/dat2.precip
    
    # some values
    R_P_min = dat2.R_P.min()
    R_P_max = dat2.R_P.max()
    P = dat2.precip.mean()
    Q = dat2.streamflow.mean() 
    L = dat2.leakage.mean()
    SWE = dat.groupby(by='wateryear').max().snowpack.mean()
    
    
    return [track,scenario,year,decade,name,idx,R_P,LAI,R_P_min,R_P_max,Q,L,P,SWE,ET,leakage,sfvals,etvals,
            transvals,subvals,snowfallvals,precipvals,evapvals,leakagevals,rzs,petvals]

In [50]:
df = pd.DataFrame({'file':glob.glob('/RHESSys/Como/out/exp/*_basin.daily')})

In [51]:
len(df)

126

In [52]:
res = df.apply(process,axis=1)

In [53]:
names = ['track','scenario','year','decade','name','idx','R_P','LAI','R_Pmin','R_Pmax','Q','L','P','SWE','ET',
         'leakage','streamflow','et','trans','snow_sub','snowfall','precip','evap','leakage','root_storage','pet']
for name,val in zip(names,zip(*res)):
    df[name] = val

In [54]:
df.sort_values('idx',inplace=True)

In [55]:
# removed b/c full run with all the climate futures do not have the correct index numbers, but those numbers are not important
# add the columns from the experiment parameter file
df['krumlai'] = dat['krumlai'].as_matrix()
df['krumweight'] = dat['krumweight'].as_matrix()

In [56]:
del df['file']

In [57]:
df.reset_index(inplace=True)

In [58]:
df.to_pickle('./data/experiment_results_v5.pcl')

In [59]:
! ls -ltrh ./data/experiment_results_*.pcl

-rwx------  1 barnhatb  staff   8.7M Oct 17 14:36 ./data/experiment_results_v2.pcl
-rwx------  1 barnhatb  staff    17M Oct 20 08:50 ./data/experiment_results_v3.pcl
-rwx------  1 barnhatb  staff    49M Oct 25 17:25 ./data/experiment_results_v4.pcl
-rwx------  1 barnhatb  staff    70M Nov  7 17:04 ./data/experiment_results_v5.pcl


In [32]:
df.year.unique()

array([2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])